<a href="https://colab.research.google.com/github/elaisemsunny/recommendar-system/blob/main/recommendar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline
 
plt.style.use('bmh')
df1=pd.read_csv('gdrive/My Drive/colab/users.csv')
df2=pd.read_csv('gdrive/My Drive/colab/posts.csv')
df3=pd.read_csv('gdrive/My Drive/colab/views.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df1.head()

,_id,name,gender,academics
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate
1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate
2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate
3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate
4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate


In [ ]:
df2.head()

,_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog


In [ ]:
df3.head()

,user_id,post_id,timestamp
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,2020-06-01T08:10:23.880Z
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z


In [ ]:
##merging the 3 datasets with respect to post_id and user_id

In [ ]:
d1=pd.merge(df3, df1, left_on="user_id", right_on="_id").drop('_id', axis=1)
d1.head()

,user_id,post_id,timestamp,name,gender,academics
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z,Niriksha Sharma,female,undergraduate
1,5df49b32cc709107827fb3c7,5ecd5d417023451e662235c5,2020-05-26T20:11:07.153Z,Niriksha Sharma,female,undergraduate
2,5df49b32cc709107827fb3c7,5dee9b5042a8854bf6eabaaf,2020-05-25T09:58:52.351Z,Niriksha Sharma,female,undergraduate
3,5df49b32cc709107827fb3c7,5ecb72c0eaff6b0c3a58a48e,2020-05-25T07:27:20.699Z,Niriksha Sharma,female,undergraduate
4,5df49b32cc709107827fb3c7,5ec3c09274f7660d73aa1229,2020-05-25T07:26:57.220Z,Niriksha Sharma,female,undergraduate


In [ ]:
df=pd.merge(d1, df2, left_on="post_id", right_on="_id").drop('_id', axis=1)
df.head()

,user_id,post_id,timestamp,name,gender,academics,title,category,post_type
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z,Niriksha Sharma,female,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
1,5d7c994d5720533e15c3b1e9,5ec821ddec493f4a2655889e,2020-05-22T20:11:32.317Z,Varun Chowhan,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
2,5e5af599d701ab08af792b63,5ec821ddec493f4a2655889e,2020-05-22T20:01:37.309Z,Ilupeju Ayokunnumi,female,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
3,5de50d768eab6401affbb135,5ec821ddec493f4a2655889e,2020-05-22T20:10:41.100Z,thesocialcomment,male,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
4,5deeef6142a8854bf6eabab9,5ec821ddec493f4a2655889e,2020-05-22T20:08:45.061Z,siddharth saxena,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork


In [ ]:
##creating a recommendition system

In [ ]:
##to recommend posts similar to a particular post
 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['title'])
 
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) 
results = {}
for idx, row in df.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
   similar_items = [(cosine_similarities[idx][i], df['post_id'][i]) for i in similar_indices] 
   results[row['post_id']] = similar_items[1:]
 
def item(id):  
  return df.loc[df['post_id'] == id]['title'].tolist()[0].split(' - ')[0] 
# Just reads the results out of the dictionary.def 
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")   
    print("-------")    
    recs = results[item_id][:num]   
    for rec in recs: 
       print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")

In [ ]:
recommend(item_id='5e8164f9a3258347b42f21bf', num=10)

Recommending 10 products similar to Is The Ozone Layer Healing Itself Because Of The World Lockdown....
-------
Recommended: Is The Ozone Layer Healing Itself Because Of The World Lockdown. (score:0.9999999999999997)
Recommended: Is The Ozone Layer Healing Itself Because Of The World Lockdown. (score:0.9999999999999997)
Recommended: Hello world (score:0.1344370987478994)
Recommended: Hello world (score:0.1344370987478994)
Recommended: Hello world (score:0.1344370987478994)
Recommended: Hello world (score:0.1344370987478994)
Recommended: Hello world (score:0.1344370987478994)
Recommended: Hello world (score:0.1344370987478994)
Recommended: Best place in the world (score:0.08327064669809252)
Recommended: CYBER WORLD WAR (score:0.07941142813884003)


In [ ]:
##to recommend similar posts to users

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['title'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) 
results = {}
for idx, row in df.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
   similar_items = [(cosine_similarities[idx][i], df['user_id'][i]) for i in similar_indices] 
   results[row['user_id']] = similar_items[1:]

def item(id):  
  return df.loc[df['user_id'] == id]['title'].tolist()[0].split(' - ')[0] 
# Just reads the results out of the dictionary.def 
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")   
    print("-------")    
    recs = results[item_id][:num]   
    for rec in recs: 
       print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")

In [ ]:
recommend(item_id='5de50d768eab6401affbb135', num=10)

Recommending 10 products similar to Save Earth....
-------
Recommended: Save Earth. (score:1.0)
Recommended: Cloud Computing (score:1.0)
Recommended: Cloud Computing (score:1.0)
Recommended: Save Earth. (score:0.5527572414069141)
Recommended: Future Communication Predictions (score:0.5527572414069141)
Recommended: Machine Learning”&“Operations" (MlOps) (score:0.5527572414069141)
Recommended: Learning... (score:0.5527572414069141)
Recommended: Machine Learning”&“Operations" (MlOps) (score:0.5527572414069141)
Recommended: Cloud Computing (score:0.5527572414069141)
Recommended: Machine Learning”&“Operations" (MlOps) (score:0.5527572414069141)
